In [65]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime as dt
import os

In [61]:
holidays_events = pd.read_csv("https://www.dropbox.com/s/bxyamlpevkiwwoq/holidays_events.csv?dl=1")
holidays_events["holiday_type"] = holidays_events["type"]
holidays_events.drop(["type"],axis=1,inplace=True)
oil = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1",parse_dates=['date'],index_col='date')
oil2 = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1")
sample_submission = pd.read_csv("https://www.dropbox.com/s/68jjl61x6u3klos/sample_submission.csv?dl=1")
stores = pd.read_csv("https://www.dropbox.com/s/lcxn6r9bs2exguq/stores.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/cvdo1gn7r5lu2uz/test.csv?dl=1",index_col='id')
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')
transactions = pd.read_csv("https://www.dropbox.com/s/92fij9bcwt0e0cj/transactions.csv?dl=1")

In [119]:
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')

In [68]:
ROOT = "C:/Users/grycz/Studia/Magisterka/II semestr 2021-2022/WzTUM/WTUM_12_2022/"

In [63]:
def przygotowanie_danych(df: pd.DataFrame, type, type_item):
    """
    Funkcja łącząca zbiory i zmieniająca zmienne kategoryczne na zmienne numeryczne. Funkcja jest przewidziana
    df - zbiór danych treningowych
    type - dane sklepowe ('family') / dane rodzinne ('store_nbr')
    type_item - konretny element zmiennej type np. dla type=="family" type_item=="AUTOMOTIVE"
    """
    #Łaczymy z pozostałymi zbiorami

    #Zbiór stores
    df = df.merge(stores,how="left",left_on=['store_nbr'],right_on=['store_nbr'])

    #Zbiór transactions
    df = df.merge(transactions,how="left",left_on=['date','store_nbr'],right_on=['date','store_nbr'])

    #Zbiór holidays_events
    df = df.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

    #Dodanie oil
    df = df.merge(oil2,how="left",left_on=['date'],right_on=['date'])

    #Interpolacja braków danych z oil
    df.interpolate(method ='linear', limit_direction ='backward', inplace=True)

    #Wybieramy family ze zbioru traningowego
    df_fam = df.loc[(df[type]==type_item)]
    
    #Dodajemy zmienne na dzień tygodnia i na miesiąc
    df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
    df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month

    #Usunięcie zmiennych, które nie będą zmieniane
    df_fam.drop([type,"description","transferred"],axis=1,inplace=True)
    if type == "family":
        type_opposite = "store_nbr"
    elif type == "store_nbr":
        type_opposite = "family"

    #One Hot Encoding
    df_fam = pd.get_dummies(df_fam,columns=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"],prefix=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"])
    
    #W zbiorze test jest tylko jeden miesiąc dlatego musimy zrobić technicnzy zabieg polegający na dodaniu kolumn z samymi zerami.
    if 'sales' not in df_fam.columns:
        missing_cols = ['sales','locale_National','locale_Regional', 'locale_name_Cayambe', 'locale_name_Cotopaxi', 'locale_name_Cuenca', 'locale_name_Ecuador', 'locale_name_El Carmen', 'locale_name_Esmeraldas', 'locale_name_Guaranda', 'locale_name_Guayaquil', 'locale_name_Ibarra', 'locale_name_Imbabura', 'locale_name_Latacunga', 'locale_name_Libertad', 'locale_name_Loja', 'locale_name_Machala', 'locale_name_Manta', 'locale_name_Puyo', 'locale_name_Quevedo', 'locale_name_Quito', 'locale_name_Riobamba', 'locale_name_Salinas', 'locale_name_Santa Elena', 'locale_name_Santo Domingo', 'locale_name_Santo Domingo de los Tsachilas', 'holiday_type_Additional', 'holiday_type_Bridge', 'holiday_type_Event', 'holiday_type_Transfer', 'holiday_type_Work Day']
        for i in missing_cols:
            df_fam[i] = 0

    return(df_fam)

In [70]:
t = przygotowanie_danych(test,"family","AUTOMOTIVE")
t.drop(["sales"],axis=1,inplace=True)


C:\Users\grycz\AppData\Local\Temp\ipykernel_22260\3479940823.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\grycz\AppData\Local\Temp\ipykernel_22260\3479940823.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\grycz\AppData\Local\Temp\ipykernel_22260\3479940823.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [71]:
t['date']

,date,onpromotion,transactions,dcoilwtico,dayofweek,month,locale_Local,store_nbr_1,store_nbr_2,store_nbr_3,...,locale_name_Riobamba,locale_name_Salinas,locale_name_Santa Elena,locale_name_Santo Domingo,locale_name_Santo Domingo de los Tsachilas,holiday_type_Additional,holiday_type_Bridge,holiday_type_Event,holiday_type_Transfer,holiday_type_Work Day
0,2017-08-16,0,NaN,46.80,3,8,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33,2017-08-16,0,NaN,46.80,3,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66,2017-08-16,0,NaN,46.80,3,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,2017-08-16,0,NaN,46.80,3,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,2017-08-16,0,NaN,46.80,3,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28347,2017-08-31,0,NaN,47.26,4,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28380,2017-08-31,0,NaN,47.26,4,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28413,2017-08-31,0,NaN,47.26,4,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28446,2017-08-31,0,NaN,47.26,4,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
temp = test.loc[:,'date'].values
np.unique(temp)

16

In [15]:
automotive = pd.read_csv("predict_2022_05_18-22-13-14/AUTOMOTIVE.csv")

In [79]:
automotive['date'] = t['date'].reset_index(drop=True)

In [83]:
train['store_nbr'] = 'store_nbr_' + str(train['store_nbr'])

In [103]:
automotive['store_nbr'] = automotive['store_nbr'].str[10:].astype(int)

In [108]:
automotive = automotive.rename(columns={'store_nbr':'sales'})

In [105]:
automotive[['sales_pred','store_nbr','date']]

,sales_pred,store_nbr,date
0,3.230153,1,2017-08-16
1,3.223934,10,2017-08-16
2,9.868250,11,2017-08-16
3,2.420875,12,2017-08-16
4,3.371859,13,2017-08-16
...,...,...,...
859,0.664927,54,2017-08-31
860,3.330138,6,2017-08-31
861,4.596535,7,2017-08-31
862,6.281710,8,2017-08-31


In [110]:
train

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,2013-01-01,1,BABY CARE,0.000,0
2,2013-01-01,1,BEAUTY,0.000,0
3,2013-01-01,1,BEVERAGES,0.000,0
4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0
3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,2017-08-15,9,PRODUCE,2419.729,148


In [111]:
train_automotive = train.loc[(train['family'] == 'AUTOMOTIVE')]

In [112]:
train_automotive

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
33,2013-01-01,10,AUTOMOTIVE,0.0,0
66,2013-01-01,11,AUTOMOTIVE,0.0,0
99,2013-01-01,12,AUTOMOTIVE,0.0,0
132,2013-01-01,13,AUTOMOTIVE,0.0,0
...,...,...,...,...,...
3000723,2017-08-15,54,AUTOMOTIVE,8.0,0
3000756,2017-08-15,6,AUTOMOTIVE,7.0,0
3000789,2017-08-15,7,AUTOMOTIVE,5.0,0


In [117]:
temp = pd.concat(train_automotive[['sales','store_nbr','date']],automotive)

C:\Users\grycz\AppData\Local\Temp\ipykernel_22260\4003793202.py:1: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [22]:
temp = []
for i in range(1,55):
    temp.append('store_nbr_'+str(i))

In [34]:

automotive['store_nbr'] =automotive.loc[:,temp].idxmax(1)

In [37]:
automotive.drop(temp,axis=1)

,onpromotion,transactions,dcoilwtico,dayofweek,month,locale_Local,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,...,locale_name_Santa Elena,locale_name_Santo Domingo,locale_name_Santo Domingo de los Tsachilas,holiday_type_Additional,holiday_type_Bridge,holiday_type_Event,holiday_type_Transfer,holiday_type_Work Day,sales_pred,store_nbr
0,0,NaN,46.80,3,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.230153,store_nbr_1
1,0,NaN,46.80,3,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.223934,store_nbr_10
2,0,NaN,46.80,3,8,0,0,0,1,0,...,0,0,0,0,0,0,0,0,9.868250,store_nbr_11
3,0,NaN,46.80,3,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.420875,store_nbr_12
4,0,NaN,46.80,3,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.371859,store_nbr_13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,NaN,47.26,4,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.664927,store_nbr_54
860,0,NaN,47.26,4,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.330138,store_nbr_6
861,0,NaN,47.26,4,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.596535,store_nbr_7
862,0,NaN,47.26,4,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6.281710,store_nbr_8


Wykres liczby sprzedanych artykułów w zależności od daty i w podziale na rodzaj artykułu

In [ ]:
fig = px.line(train, x='date', y='sales', color='family')
fig.show()

Wykres liczby sprzedanych artykułów w zależności od daty i w podziale na numer sklepu

In [ ]:
fig = px.line(train, x='date', y='sales', color='store_nbr')
fig.show()